# Assignment 2 (Stage Classification of Liver Cancer)

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, chi2, f_regression
from sklearn.preprocessing import Normalizer

## To read Train and Test datasets

In [3]:
data_train=pd.read_csv('train.csv')
data_test=pd.read_csv("test.csv")

# data=pd.concat(data)
# print(data_train.head(5))
# print(data_test.head(5))
print (data_train.shape)
print (data_test.shape)
data_train_lable=data_train['Label']
data_train.drop(['Label','id'],axis=1,inplace=True)
data_test_ID=data_test['id']
data_test.drop(['id'],axis=1,inplace=True)

# data_train,data_train_lable=shuffle(data_train,data_train_lable)
# print (data_train.columns)
# print(data_test.columns)
print((data_train.shape),data_train_lable.shape,(data_test.shape))

# data=pd.concat([data_train,data_test])
# print(data.shape)
print(data_test_ID.shape)


(279, 60485)
(71, 60484)
(279, 60483) (279,) (71, 60483)
(71,)


## code for Normalization

In [4]:
# transformer = Normalizer().fit(data_train)
# data_train=pd.DataFrame(transformer.transform(data_train))
# transformer = Normalizer().fit(data_test)
# data_test=pd.DataFrame(transformer.transform(data_test))
# X_train=data_train
# y_train=data_train_lable
# X_test=data_test
# transformer = Normalizer().fit(data_train)
# data_train=pd.DataFrame(transformer.transform(data_train))
# transformer = Normalizer().fit(data_test)
# data_test=pd.DataFrame(transformer.transform(data_test))

# Feature selection till no of feature remains 7 using SelectKBest

In [5]:

# pca = PCA(n_components=100)
# data = pca.fit_transform(data)
# X_train, X_test, y_train, y_test = train_test_split(data_train, data_train_lable, test_size=0.2)
X_train=data_train
y_train=data_train_lable
X_test=data_test
select_feature = SelectKBest(chi2, k=7).fit(X_train, y_train)
selected_features_df = pd.DataFrame({'Feature':list(X_train.columns),
                                     'Scores':select_feature.scores_})
selected_features_df.sort_values(by='Scores', ascending=False)
X_train_chi = pd.DataFrame(select_feature.transform(X_train))

X_test_chi = pd.DataFrame(select_feature.transform(X_test))
print(X_train_chi.columns,X_test_chi.columns)

RangeIndex(start=0, stop=7, step=1) RangeIndex(start=0, stop=7, step=1)


# GridSearchCV for finding best parameter configuration for SVC classifier

In [9]:
def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    kernels=['linear','rbf']
    param_grid = {'C': Cs, 'gamma' : gammas,'kernel':kernels}
    grid_search = GridSearchCV(svm.SVC(), param_grid, cv=nfolds,scoring = 'roc_auc')
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_,grid_search
params,clf=svc_param_selection(X_train_chi, y_train, 5)
print(params)
print("Cross Validation AUC Score:",clf.best_score_)

{'C': 0.01, 'gamma': 0.001, 'kernel': 'linear'}
Cross Validation AUC Score: 0.7162751545089945


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


<b> using the classifier which is working optimally sometimes it may change because of some random states</b>

In [10]:
clf=svm.SVC(C= 1, gamma= 0.1, kernel= 'rbf')
clf.fit(X_train_chi,y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

## Predict and store the result of the model

In [11]:
predictions=clf.predict(X_test_chi)
# df_merged = pd.merge(data_test_ID, pd.DataFrame(predictions ))
# df_merged['ID'] =data_test_ID
# data_test_ID=data_test_IDpredictions
# print(data_test_ID.head())
# print(predictions.shape,data_test_ID.shape)
prediction = pd.DataFrame({'Id':data_test_ID,'Label': predictions})
# print(prediction)
# pd.to_csv('check.csv')
prediction.to_csv (r'output_dataframe_new_svm.csv', index = False, header=True)